# Liblary and generate data

In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
import random
import seaborn as sns
import warnings
warnings.simplefilter('ignore')
%matplotlib inline

In [ ]:
def gendata_poly_sin(n,d,min,max,w,sigma):
    x = np.zeros((n,d+1))
    x_1 = np.sort(np.random.rand(n)*(max-min)+min)
    for i in range(1,d+1):
        x[:,i]= x_1**i
        
    y = np.zeros(n)
    
    for i in range(n):
        y[i] = w@x[i,:]+math.sin(x[i,1]*4)*1 +np.random.randn()*sigma
    return x[:, 1],y

# DNN(From Scratch)

In [ ]:
class sigmoid():
    def forward(self,h):
        return 1/(1 + np.exp(-h))
    
    def backward(self,x):
        return (1.0 - self.forward(x)) * self.forward(x)
    
class relu():
    def forward(self, h):
        return h * (h > 0.0)
    
    def backward(self, x):
        return 1.0 * (x > 0.0)

class MSE():
    def forward(self, x, y):
        return np.square(x-y)
    
    def backward(self, x, y):
        return 2*(x- y)
    
class DNN:

    def __init__(self, input_size, hidden_size, output_size,acti, loss, lr=0.001):
        
        self.input_size=input_size
        self.output_size=output_size
        self.acti=acti
        self.loss=loss
        self.lr=lr
        self.params = {}
        #initialization of parameters 
        self.params['W1'] = np.random.randn(self.input_size, hidden_size) / np.sqrt(input_size)
        self.params['b1'] = np.zeros(hidden_size) 
        self.params['W2'] = np.random.randn(hidden_size, output_size)/ np.sqrt(hidden_size)
        self.params['b2'] = np.zeros(output_size)
    
        
    def forward(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        
        self.params["A0"]=x
        #print(W1.shape, x.shape)
        self.params["a1"] = np.dot(self.params["A0"], W1) + b1
        self.params["z1"] = self.acti.forward(self.params["a1"])
        self.params["a2"] = np.dot(self.params["z1"], W2) + b2
        #print(W2.shape)
        self.params["z2"] = self.params["a2"]
        
        return self.params["z2"].ravel()
    
    def backward(self, t, output):
        grads = {}
        dy = self.loss.backward(x=output,y=t)
        #print(dy)
        grads['W2'] = np.dot(self.params["z1"].T, dy).reshape(-1, 1)
        #print(self.params["z2"].shape)
        grads['b2'] = np.sum(dy, axis=0)
        
        dz1 = np.dot(dy, self.params['W2'].T)
        da1 = (self.acti.backward(self.params["a1"]) * dz1)
        grads['W1'] = np.dot(self.params["A0"].T, da1)
        #print(np.dot(self.params["A0"].T, da1))
        grads['b1'] = np.sum(da1, axis=0)

        return grads
    
    
    def update(self, grads):
        for key in ('W1', 'b1', 'W2', 'b2'):
            #print(key)
            #print(self.params[key].shape,  grads[key].shape)
            self.params[key] -= self.lr * grads[key]
       
    #predict
    def predict(self, x):
        output = self.forward(x)
        return output
    
    #calc loss
    def loss_(self, t, output):
        loss = self.loss.forward(output,t)
        return loss
        
            
    #dispkay the layer structure
    def display(self):
        print(f"the size of the input: (None, {self.input_size})")
        print(f"hidden1: input(None, {self.input_size}),output({self.params['W1'].shape})")
        print(f"hidden2: input({self.params['W1'].shape}),output({self.params['W2'].shape})")
        print(f"the size of the output: (None, {self.output_size})")
    #for debug
    def params(self):
        return self.params

In [ ]:
from sklearn.model_selection import train_test_split
x_train, y_train=gendata_poly_sin(1000,6,0,10,[0,-2,1,0,0,0,0],0.5)
x_test, y_test = gendata_poly_sin(300,6,0,10,[0,-2,1,0,0,0,0],0.9)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

network = DNN(input_size=1, hidden_size=100, output_size=1, acti=sigmoid(), loss=MSE())
network.display()

In [ ]:
history=[] #the history of loss
iters_num = 100 # the number of the epoch
bestloss=np.inf # recording the lowest loss at the time
for i in range(iters_num):
    train_byepoch_list = []
    for x, y in zip(x_train,y_train):
        output = network.forward(x)#forward
        grads = network.backward(y, output)#backward
        network.update(grads)#update paramters
        loss = network.loss_(y, output)# calc loss
        train_byepoch_list.append(loss)
        
        if loss < bestloss:
            bestloss=loss
            
    history.append(np.mean(train_byepoch_list))
    
    if i % 10 ==0:# display log
        print("epochs:", i, "======== loss:", np.mean(train_byepoch_list))  
print("The best train loss: ",bestloss) 

#predict
pred=[]
for test in zip(x_test):
    pred.append(network.predict(test))

print("Test MSE:", network.loss_(y_test, np.array(pred)).mean())
    

In [ ]:
plt.plot(x_train,y_train, label="train")
plt.plot(x_test,y_test, label="test")
plt.plot(x_test, pred, label="pred")
plt.legend(bbox_to_anchor=(1, 1), loc='Upper Left', borderaxespad=0, fontsize=12)
plt.show()

# DNN(Keras)

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from keras.optimizers import SGD, Adam
from keras.optimizers import Optimizer
from keras import backend as K

In [ ]:
#SGD(not work well)
model = keras.Sequential()
model.add(layers.Dense(100, input_dim=1, activation='relu', kernel_initializer='random_normal',bias_initializer='zeros'))
model.add(layers.Dense(1, kernel_initializer='random_normal',bias_initializer='zeros'))
model.compile(optimizer=SGD(momentum=0.0, learning_rate=0.001), loss='mean_squared_error')
model.summary()

batch_size = 1
epochs = 100

keras_history=model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs)
print("Learning done!")

from sklearn.metrics import mean_squared_error
pred_keras=model.predict(x_test)
print("Test MSE: ", mean_squared_error(y_test, pred_keras))

plt.plot(x_train,y_train, label="train")
plt.plot(x_test,y_test, label="test")
plt.plot(x_test, pred_keras, label="pred")
plt.legend(bbox_to_anchor=(1, 1), loc='Upper Left', borderaxespad=0, fontsize=12)
plt.show()

In [ ]:
#Adam version
model = keras.Sequential()
model.add(layers.Dense(100, input_dim=1, activation='relu', kernel_initializer='random_normal',bias_initializer='zeros'))
model.add(layers.Dense(1, kernel_initializer='random_normal',bias_initializer='zeros'))
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
model.summary()

batch_size = 1
epochs = 100

keras_history=model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs)
print("Learning done!")

from sklearn.metrics import mean_squared_error
pred_keras=model.predict(x_test)
print("Test MSE: ", mean_squared_error(y_test, pred_keras))

plt.plot(x_train,y_train, label="train")
plt.plot(x_test,y_test, label="test")
plt.plot(x_test, pred_keras, label="pred")
plt.legend(bbox_to_anchor=(1, 1), loc='Upper Left', borderaxespad=0, fontsize=12)
plt.show()